#### Author: WEE LING HUE

In [1]:
import nltk
print(nltk.data.path)


['/home/student/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [2]:
import nltk
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/student/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
#Import Necessary Libraries

from pyspark.sql import SparkSession
from sentimentml.text_preprocessor import text_preprocessor
from sentimentml.sentiment_model_trainer import sentiment_model_trainer
from sentimentml.text_cleaner import text_cleaner
from sentimentml.eng_word_lemmatizer import eng_word_lemmatizer
from sentimentml.malay_word_stemmer import malay_word_stemmer
from pyspark.sql.functions import col, sum


In [2]:
# Initialize Spark Session
spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()

25/04/18 18:17:03 WARN Utils: Your hostname, PC21. resolves to a loopback address: 127.0.1.1; using 172.26.221.164 instead (on interface eth0)
25/04/18 18:17:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/18 18:17:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load Data

In [3]:
# Path to HDFS
hdfs_path = "hdfs:/user/hduser/raw_tweets/"

# Step 1: Read JSON data from HDFS with handling for corrupted records
df = spark.read.option("mode", "PERMISSIVE").json(hdfs_path)

# Step 2: Show the schema to inspect how the data is structured
df.printSchema()

# Step 3: Show some raw data to check the actual structure
df.show(5, truncate=False)

# Step 4: Drop the '_corrupt_record' column if it exists
df = df.drop('_corrupt_record')

# Step 5: If the data is nested, flatten it using selectExpr to map the fields correctly
# Let's assume the data might be nested. You can adjust column names accordingly if needed
df = df.selectExpr("`User ID`", "`Name`", "`Followers Count`", "`Tweet`", "`Location`", "`Tweet Time`", "`Friends Count`", "`Sentiment`")

# Step 6: Rename columns properly
correct_columns = ["User ID", "Name", "Followers Count", "Tweet", "Location", "Tweet Time", "Friends Count", "Sentiment"]
df = df.toDF(*correct_columns)

# Step 7: Show top 5 rows and print row count
print(f"Total rows: {df.count()}")
df.show(5, truncate=False)


root
 |-- Followers Count: long (nullable = true)
 |-- Friends Count: long (nullable = true)
 |-- Location: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- Tweet: string (nullable = true)
 |-- Tweet Time: string (nullable = true)
 |-- User ID: long (nullable = true)
 |-- _corrupt_record: string (nullable = true)

+---------------+-------------+----------------------+--------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------+--------+---------------+
|Followers Count|Friends Count|Location              |Name    |Sentiment|Tweet                                                                                                                                                               |Tweet Time                    |User ID |_corrupt_record|
+---------------+-------------+-----

## Check Data

In [4]:
from pyspark.sql import functions as F

# Step 1: Check for null values in each column
null_counts = df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Show null counts
null_counts.show()

# Step 2: Remove rows with null values
df_cleaned = df.dropna()

# Step 3: Check for duplicate rows
duplicate_count = df.count() - df.dropDuplicates().count()
print(f"Duplicate Rows: {duplicate_count}")

# Step 4: Remove duplicate rows
df_cleaned = df_cleaned.dropDuplicates()

# Step 5: Show first 5 rows after cleaning
df_cleaned.show(5, truncate=False)

# Step 6: Print total number of records after cleaning
print(f"Total Records After Cleaning: {df_cleaned.count()}")

# Step 7: Count records by sentiment label
sentiment_counts = df_cleaned.groupBy("Sentiment").count()

# Show sentiment counts
sentiment_counts.show()


+-------+----+---------------+-----+--------+----------+-------------+---------+
|User ID|Name|Followers Count|Tweet|Location|Tweet Time|Friends Count|Sentiment|
+-------+----+---------------+-----+--------+----------+-------------+---------+
|      2|   2|              2|    2|       2|         2|            2|        2|
+-------+----+---------------+-----+--------+----------+-------------+---------+

Duplicate Rows: 1
+--------+-----------------+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+------------------------------+-------------+---------+
|User ID |Name             |Followers Count|Tweet                                                                                                                                                                           

## Preprocessing Data

In [5]:
# Split dataset into training (80%) and testing (20%)
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

from sentimentml.text_preprocessor import text_preprocessor

# Initialize the text preprocessing pipeline
text_preprocessor = text_preprocessor(input_col="Tweet", label_col="Sentiment")\

# Get the preprocessing pipeline
pipeline = text_preprocessor.get_pipeline()

In [6]:
pipeline_model = pipeline.fit(train_data)

25/04/18 18:17:16 ERROR PythonUDFRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/hduser/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1225, in main
    eval_type = read_int(infile)
  File "/home/hduser/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 596, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:94)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:594)
	at scala.collection.Iterator

Py4JJavaError: An error occurred while calling o107.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 1 times, most recent failure: Lost task 0.0 in stage 33.0 (TID 20) (172.26.221.164 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`Tokenizer$$Lambda$3790/922709059`: (string) => array<string>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.ScalaUDF_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$10(EvalPythonExec.scala:127)
	at scala.coll

In [9]:
# Fit and transform the training data

processed_train_data = pipeline_model.transform(train_data)

25/04/18 18:15:15 ERROR PythonUDFRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/hduser/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1225, in main
    eval_type = read_int(infile)
  File "/home/hduser/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 596, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:94)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:594)
	at scala.collection.Iterator

Py4JJavaError: An error occurred while calling o107.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 1 times, most recent failure: Lost task 0.0 in stage 33.0 (TID 20) (172.26.221.164 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`Tokenizer$$Lambda$3790/976852428`: (string) => array<string>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.ScalaUDF_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$10(EvalPythonExec.scala:127)
	at scala.coll

In [ ]:






# Transform the test data using the same pipeline model
processed_test_data = pipeline_model.transform(test_data)

# Show processed data
processed_train_data.select("Tweet", "clean_text", "words", "filtered_words", "lemmatized_words", "stemmed_words", "features", "label").show(10, truncate=False)

In [5]:
from pyspark.sql import SparkSession
from sentimentml.text_preprocessor import text_preprocessor  # Importing the preprocessing class

# Step 1: Create Spark session
spark = SparkSession.builder.appName("TextPreprocessing").getOrCreate()

# Step 2: Read the raw dataset
df = spark.read.csv("file:///home/username/de-prj/tweets_output_with_sentiment.csv", header=True, inferSchema=True)

# Step 3: Clean the data (removing empty or null tweets)
df_cleaned = df.filter(df["Tweet"].isNotNull() & (df["Tweet"] != ""))

# Step 4: Split dataset into training (80%) and testing (20%)
train_data, test_data = df_cleaned.randomSplit([0.8, 0.2], seed=42)

# Step 5: Initialize the text preprocessing pipeline
text_preprocessor_step = text_preprocessor(input_col="Tweet", label_col="Sentiment")

# Step 6: Get the preprocessing pipeline
pipeline = text_preprocessor_step.get_pipeline()

# Step 7: Fit and transform the training data
pipeline_model = pipeline.fit(train_data)
processed_train_data = pipeline_model.transform(train_data)

# Step 8: Transform the test data using the same pipeline model
processed_test_data = pipeline_model.transform(test_data)

# Step 9: Show the processed data
processed_train_data.select("Tweet", "clean_text", "words", "filtered_words", "lemmatized_words", "stemmed_words", "features", "label").show(10, truncate=False)


25/04/18 18:11:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/username/de-prj/tweets_output_with_sentiment.csv.

In [5]:
#nltk.download('averaged_perceptron_tagger')


NameError: name 'nltk' is not defined

In [6]:
#import os
#os.environ['NLTK_DATA'] = '/home/student/nltk_data'  


In [7]:
from pyspark.sql import SparkSession
from sentimentml.text_preprocessor import text_preprocessor
from pyspark.sql.functions import col

# Initialize Spark session and context
spark = SparkSession.builder \
    .appName("Text Preprocessing") \
    .getOrCreate()
sc = spark.sparkContext  # Get SparkContext

# Broadcast NLTK data
sc.addFile('/home/student/nltk_data')  # Path to NLTK data

# Step 1: Check for and remove empty or null tweets
df_cleaned = df.filter(df["Tweet"].isNotNull() & (df["Tweet"] != ""))

# Step 2: Proceed with the same pipeline
# Split dataset into training (80%) and testing (20%)
train_data, test_data = df_cleaned.randomSplit([0.8, 0.2], seed=42)

# Initialize the text preprocessing pipeline
text_preprocessor_step = text_preprocessor(input_col="Tweet", label_col="Sentiment")

# Get the preprocessing pipeline
pipeline = text_preprocessor_step.get_pipeline()

# Fit and transform the training data
pipeline_model = pipeline.fit(train_data)
processed_train_data = pipeline_model.transform(train_data)

# Transform the test data using the same pipeline model
processed_test_data = pipeline_model.transform(test_data)

# Show processed data
processed_train_data.select("Tweet", "clean_text", "words", "filtered_words", "lemmatized_words", "stemmed_words", "features", "label").show(10, truncate=False)


25/04/18 17:58:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Py4JJavaError: An error occurred while calling o97.addFile.
: org.apache.spark.SparkException: [UNSUPPORTED_ADD_FILE.DIRECTORY] Don't support add file. The file file:/home/student/nltk_data/ is a directory, consider to set "spark.sql.legacy.addSingleFileInAddFile" to "false".
	at org.apache.spark.errors.SparkCoreErrors$.addDirectoryError(SparkCoreErrors.scala:490)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1760)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1733)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractComman

In [6]:
# Step 1: Check for and remove empty or null tweets
df_cleaned = df.filter(df["Tweet"].isNotNull() & (df["Tweet"] != ""))

# Step 2: Proceed with the same pipeline
# Split dataset into training (80%) and testing (20%)
train_data, test_data = df_cleaned.randomSplit([0.8, 0.2], seed=42)

# Initialize the text preprocessing pipeline
text_preprocessor_step = text_preprocessor(input_col="Tweet", label_col="Sentiment")

# Get the preprocessing pipeline
pipeline = text_preprocessor_step.get_pipeline()

# Fit and transform the training data
pipeline_model = pipeline.fit(train_data)
processed_train_data = pipeline_model.transform(train_data)

# Transform the test data using the same pipeline model
processed_test_data = pipeline_model.transform(test_data)

# Show processed data
processed_train_data.select("Tweet", "clean_text", "words", "filtered_words", "lemmatized_words", "stemmed_words", "features", "label").show(10, truncate=False)


25/04/18 18:11:58 ERROR Executor: Exception in task 0.0 in stage 33.0 (TID 20)1]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/student/de-prj/sentimentml/eng_word_lemmatizer.py", line 30, in nltk_lemmatizer
    return [lemmatizer.lemmatize(word, eng_word_lemmatizer.get_wordnet_pos(word)) for word in words]
  File "/home/student/de-prj/sentimentml/eng_word_lemmatizer.py", line 30, in <listcomp>
    return [lemmatizer.lemmatize(word, eng_word_lemmatizer.get_wordnet_pos(word)) for word in words]
  File "/home/student/de-prj/sentimentml/eng_word_lemmatizer.py", line 22, in get_wordnet_pos
    tag = pos_tag([word])[0][1][0].upper()
  File "/home/student/de-prj/de-venv/lib/python3.10/site-packages/nltk/tag/__init__.py", line 168, in pos_tag
    tagger = _get_tagger(lang)
  File "/home/student/de-prj/de-venv/lib/python3.10/site-packages/nltk/tag/__init__.py", line 110, in _get_tagger
    tagger = PerceptronTagger()
  File "/home/student/de-prj/d

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/student/de-prj/sentimentml/eng_word_lemmatizer.py", line 30, in nltk_lemmatizer
    return [lemmatizer.lemmatize(word, eng_word_lemmatizer.get_wordnet_pos(word)) for word in words]
  File "/home/student/de-prj/sentimentml/eng_word_lemmatizer.py", line 30, in <listcomp>
    return [lemmatizer.lemmatize(word, eng_word_lemmatizer.get_wordnet_pos(word)) for word in words]
  File "/home/student/de-prj/sentimentml/eng_word_lemmatizer.py", line 22, in get_wordnet_pos
    tag = pos_tag([word])[0][1][0].upper()
  File "/home/student/de-prj/de-venv/lib/python3.10/site-packages/nltk/tag/__init__.py", line 168, in pos_tag
    tagger = _get_tagger(lang)
  File "/home/student/de-prj/de-venv/lib/python3.10/site-packages/nltk/tag/__init__.py", line 110, in _get_tagger
    tagger = PerceptronTagger()
  File "/home/student/de-prj/de-venv/lib/python3.10/si

## Save Preprocessed Training & Test Data

In [ ]:
# Save processed training data
processed_train_data.write.mode("overwrite").parquet("hdfs://localhost:9000/user/student/processed_train_data")

# Save processed testing data
processed_test_data.write.mode("overwrite").parquet("hdfs://localhost:9000/user/student/processed_test_data")


In [ ]:
# Show the count of examples by sentiment for the processed training data
processed_train_data.groupBy("Sentiment").count().show()

# Show the count of examples by sentiment for the processed testing data
processed_test_data.groupBy("Sentiment").count().show()


In [ ]:
processed_train_data.select("label", "sentiment").distinct().show()

## Save features and label for model building

In [ ]:
# Path to store processed data in HDFS
hdfs_path = "hdfs://localhost:9000/user/student/preprocessed_data.parquet"

# Save the DataFrame to HDFS in Parquet format
processed_train_data.select("features", "label").write.mode("overwrite").parquet(hdfs_path)

print(f"Preprocessed data saved to {hdfs_path}")

# Start Here

In [ ]:
# Step 1: Import Necessary Libraries
from pyspark.sql import SparkSession
from sentimentml.text_preprocessor import text_preprocessor
from sentimentml.sentiment_model_trainer import sentiment_model_trainer
from sentimentml.text_cleaner import text_cleaner
from sentimentml.eng_word_lemmatizer import eng_word_lemmatizer
from sentimentml.malay_word_stemmer import malay_word_stemmer

In [ ]:
# Initialize Spark Session
spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()

# Load the preprocessed data from HDFS
loaded_data = spark.read.parquet("hdfs://localhost:9000/user/student/preprocessed_data.parquet")
# Load training and testing data from HDFS
train_data = spark.read.parquet("hdfs://localhost:9000/user/student/processed_train_data")
test_data = spark.read.parquet("hdfs://localhost:9000/user/student/processed_test_data")


# Verify the loaded data
loaded_data.show(5)

# Model Building

## 1. Naive Bayes

In [ ]:
trainer = sentiment_model_trainer()

# Model: naive_bayes
model_type = "naive_bayes"

# Train the model
trained_model = trainer.train(train_data, model_type)

#Test the model
evaluation_results = trainer.evaluate(trained_model, train_data, test_data)

# Save the trained model to HDFS
trained_model.write().overwrite().save("hdfs://localhost:9000/user/student/nb_model")

# Print confirmation message
print("Model successfully saved at hdfs://localhost:9000/user/student/nb_model")


## 2. Random Forest

In [ ]:
trainer = sentiment_model_trainer()

# Model name: random_forest
model_type = "random_forest"  

# Train the model
trained_model = trainer.train(train_data, model_type)

#Test the model
evaluation_results = trainer.evaluate(trained_model, train_data, test_data)

# Save the trained model to HDFS
trained_model.write().overwrite().save("hdfs://localhost:9000/user/student/rf_model")

# Print confirmation message
print("Model successfully saved at hdfs://localhost:9000/user/student/rf_model")

## 3. Logistic Regression

In [ ]:
trainer = sentiment_model_trainer()

# Model name: logistic_regression
model_type = "logistic_regression" 

# Train the model
trained_model = trainer.train(train_data, model_type)

#Test the model
evaluation_results = trainer.evaluate(trained_model, train_data, test_data)

# Save the trained model to HDFS
trained_model.write().overwrite().save("hdfs://localhost:9000/user/student/lr_model")

# Print confirmation message
print("Model successfully saved at hdfs://localhost:9000/user/student/lr_model")

## 4. Decision Tree

In [ ]:
trainer = sentiment_model_trainer()

# Model name: decision_tree
model_type = "decision_tree"  

# Train the model
trained_model = trainer.train(train_data, model_type)

#Test the model
evaluation_results = trainer.evaluate(trained_model, train_data, test_data)

# Save the trained model to HDFS
trained_model.write().overwrite().save("hdfs://localhost:9000/user/student/dt_model")

# Print confirmation message
print("Model successfully saved at hdfs://localhost:9000/user/student/dt_model")